In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler, LabelEncoder
import math
from tqdm import tqdm

In [ ]:
# 1. 데이터 불러오기
paths = {'data': '/kaggle/input/2025-electricity-consumption-dataset'}
train = pd.read_csv(f"{paths['data']}/train.csv")
train = train.drop(['일조(hr)', '일사(MJ/m2)'], axis=1)
test = pd.read_csv(f"{paths['data']}/test.csv")
b_info = pd.read_csv(f"{paths['data']}/building_info.csv")
sub = pd.read_csv(f"{paths['data']}/sample_submission.csv")

# 건물 정보에 구간 피처 추가
b_info['연면적구간'] = pd.cut(b_info['연면적(m2)'],
    bins=[0,15000,30000,45000,float('inf')],
    labels=['소형(<15k)','중형(15k-30k)','대형(30k-45k)','초대형(45k+)'])
b_info['냉방면적구간'] = pd.cut(b_info['냉방면적(m2)'],
    bins=[0,10000,20000,30000,float('inf')],
    labels=['소형(<10k)','중형(10k-20k)','대형(20k-30k)','초대형(30k+)'])

# 2. 건물 정보 병합
train = train.merge(b_info, on='건물번호', how='left')
test = test.merge(b_info, on='건물번호', how='left')

In [ ]:
# 3. 시간 피처 생성 함수
def add_time_features(df):
    # 날짜 변환 (이미 변환된 경우 스킵)
    if df['일시'].dtype == 'object':
        df['일시'] = pd.to_datetime(df['일시'], format='%Y%m%d %H')
    
    # 기본 시간 피처
    df['month'] = df['일시'].dt.month
    df['week_of_year'] = df['일시'].dt.isocalendar().week
    df['dow'] = df['일시'].dt.dayofweek
    df['day'] = df['일시'].dt.day
    df['hour'] = df['일시'].dt.hour
    
    # 휴일 설정
    df['is_holiday'] = df['dow'].apply(lambda x: 1 if x >= 5 else 0)
    holidays = [pd.Timestamp('2024-06-06'), pd.Timestamp('2024-08-15')]
    df.loc[df['일시'].dt.normalize().isin(holidays), 'is_holiday'] = 1
    # 평일 설정
    df['is_weekday'] = df['dow'].apply(lambda x: 1 if x < 5 else 0)
    df.loc[df['일시'].dt.normalize().isin(holidays), 'is_weekday'] = 0
    
    # 건물별 휴무일 설정
    df['is_building_holiday'] = 0
    
    # 일요일만 쉬는 건물
    sun_holiday = [18]
    if sun_holiday:
        df.loc[(df['건물번호'].isin(sun_holiday)) & (df['dow'] == 6), 'is_building_holiday'] = 1
    
    # 토/일 쉬는 건물
    sat_sun_holiday = [2, 3, 5, 6, 8, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 37, 
                      39, 42, 44, 46, 47, 48, 49, 50, 53, 55, 60, 62, 66, 68, 69, 72, 75, 80, 86, 87, 90, 94]
    if sat_sun_holiday:
        df.loc[(df['건물번호'].isin(sat_sun_holiday)) & (df['dow'] >= 5), 'is_building_holiday'] = 1
    # 토/일 + 공휴일 쉬는 건물
    public_holiday = [2, 3, 5, 6, 8, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 33, 37, 
                     39, 42, 44, 46, 47, 48, 49, 50, 51, 53, 55, 60, 62, 66, 68, 69, 72, 75, 80, 83, 86, 87, 90, 94]
    if public_holiday:
        # 토/일
        df.loc[(df['건물번호'].isin(public_holiday)) & (df['dow'] >= 5), 'is_building_holiday'] = 1
        # 공휴일
        df.loc[(df['건물번호'].isin(public_holiday)) & (df['일시'].dt.normalize().isin(holidays)), 'is_building_holiday'] = 1
    
    # 건물별 기타 휴무일
    etc_holiday = {
            7: [{"week_of_year": 27, "dow": 6}, {"week_of_year": 28, "dow": 0}, {"week_of_year": 28, "dow": 5}, 
                {"week_of_year": 28, "dow": 6}, {"week_of_year": 29, "dow": 0}, {"week_of_year": 29, "dow": 1},
                {"week_of_year": 29, "dow": 2}, {"week_of_year": 29, "dow": 3}, {"week_of_year": 29, "dow": 4},
                {"week_of_year": 29, "dow": 5}, {"week_of_year": 29, "dow": 6}, {"week_of_year": 30, "dow": 0},
                {"week_of_year": 30, "dow": 1}, {"week_of_year": 30, "dow": 2}, {"week_of_year": 30, "dow": 3},
                {"week_of_year": 30, "dow": 4}, {"week_of_year": 30, "dow": 5}, {"week_of_year": 30, "dow": 6},
                {"week_of_year": 31, "dow": 0}, {"week_of_year": 31, "dow": 1}, {"week_of_year": 31, "dow": 2},
                {"week_of_year": 31, "dow": 3}, {"week_of_year": 31, "dow": 4}, {"week_of_year": 31, "dow": 5},
                {"week_of_year": 31, "dow": 6}, {"week_of_year": 32, "dow": 0}],
            10: [{"week_of_year": 22, "dow": 5}, {"week_of_year": 22, "dow": 6}, {"week_of_year": 27, "dow": 6},
                 {"week_of_year": 23, "dow": 0}, {"week_of_year": 23, "dow": 1}, {"week_of_year": 23, "dow": 2},
                 {"week_of_year": 23, "dow": 3}, {"week_of_year": 23, "dow": 4}, {"week_of_year": 23, "dow": 5},
                 {"week_of_year": 23, "dow": 6}, {"week_of_year": 24, "dow": 0}, {"week_of_year": 24, "dow": 1},
                 {"week_of_year": 24, "dow": 2}, {"week_of_year": 24, "dow": 3}, {"week_of_year": 24, "dow": 4},
                 {"week_of_year": 24, "dow": 5}, {"week_of_year": 24, "dow": 6}, {"week_of_year": 25, "dow": 1},
                 {"week_of_year": 25, "dow": 2}, {"week_of_year": 25, "dow": 3}, {"week_of_year": 25, "dow": 4},
                 {"week_of_year": 25, "dow": 5}, {"week_of_year": 25, "dow": 6}, {"week_of_year": 26, "dow": 0},
                 {"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3},
                 {"week_of_year": 26, "dow": 4}, {"week_of_year": 26, "dow": 5}, {"week_of_year": 26, "dow": 6},
                 {"week_of_year": 27, "dow": 0}, {"week_of_year": 27, "dow": 1}, {"week_of_year": 27, "dow": 2}],
            12: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3}, {"week_of_year": 26, "dow": 4}],
            17: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}],
            19: [{"week_of_year": 24, "dow": 0}, {"week_of_year": 28, "dow": 0}, {"week_of_year": 34, "dow": 0}],
            23: [{"week_of_year": 23, "dow": 4}, {"week_of_year": 33, "dow": 4}],
            24: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3}, {"week_of_year": 26, "dow": 4}],
            27: [{"week_of_year": 23, "dow": 6}, {"week_of_year": 25, "dow": 6}, {"week_of_year": 28, "dow": 6}, 
                 {"week_of_year": 30, "dow": 6}, {"week_of_year": 32, "dow": 6}, {"week_of_year": 34, "dow": 6}],
            29: [{"week_of_year": 24, "dow": 0}, {"week_of_year": 25, "dow": 6}, {"week_of_year": 28, "dow": 2}, 
                 {"week_of_year": 30, "dow": 6}, {"week_of_year": 32, "dow": 5}],
            32: [{"week_of_year": 24, "dow": 0}, {"week_of_year": 26, "dow": 0}, {"week_of_year": 28, "dow": 0}, 
                 {"week_of_year": 30, "dow": 0}, {"week_of_year": 33, "dow": 0}, {"week_of_year": 35, "dow": 0}],
            33: [{"week_of_year": 27, "dow": 5}, {"week_of_year": 27, "dow": 6}, {"week_of_year": 28, "dow": 0}],
            38: [{"week_of_year": 23, "dow": 4}, {"week_of_year": 26, "dow": 0}],
            40: [{"week_of_year": 23, "dow": 6}, {"week_of_year": 25, "dow": 6}, {"week_of_year": 28, "dow": 6}, 
                 {"week_of_year": 30, "dow": 6}, {"week_of_year": 32, "dow": 6}, {"week_of_year": 34, "dow": 6}],
            41: [{"week_of_year": 25, "dow": 5}],
            42: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3}, {"week_of_year": 26, "dow": 4}],
            45: [{"week_of_year": 24, "dow": 0}, {"week_of_year": 28, "dow": 0}, {"week_of_year": 34, "dow": 0}],
            46: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3}, {"week_of_year": 26, "dow": 4}],
            49: [{"week_of_year": 34, "dow": 3}],
            54: [{"week_of_year": 25, "dow": 0}, {"week_of_year": 25, "dow": 1}, {"week_of_year": 27, "dow": 0}, 
                 {"week_of_year": 27, "dow": 1}, {"week_of_year": 34, "dow": 0}, {"week_of_year": 34, "dow": 1}],
            59: [{"week_of_year": 23, "dow": 6}, {"week_of_year": 25, "dow": 6}, {"week_of_year": 28, "dow": 6}, 
                 {"week_of_year": 30, "dow": 6}, {"week_of_year": 32, "dow": 6}, {"week_of_year": 34, "dow": 6}],
            60: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3}, {"week_of_year": 26, "dow": 4}],
            63: [{"week_of_year": 23, "dow": 6}, {"week_of_year": 25, "dow": 6}, {"week_of_year": 28, "dow": 6}, 
                 {"week_of_year": 30, "dow": 6}, {"week_of_year": 32, "dow": 6}, {"week_of_year": 34, "dow": 6}],
            67: [{"week_of_year": 30, "dow": 4}, {"week_of_year": 30, "dow": 5}],
            69: [{"week_of_year": 26, "dow": 1}, {"week_of_year": 26, "dow": 2}, {"week_of_year": 26, "dow": 3}, {"week_of_year": 26, "dow": 4}],
            70: [{"week_of_year": 23, "dow": 1}, {"week_of_year": 23, "dow": 2}],
            74: [{"week_of_year": 23, "dow": 6}, {"week_of_year": 25, "dow": 0}, {"week_of_year": 25, "dow": 6}, 
                 {"week_of_year": 27, "dow": 0}, {"week_of_year": 28, "dow": 6}, {"week_of_year": 30, "dow": 6},
                 {"week_of_year": 32, "dow": 6}, {"week_of_year": 34, "dow": 6}],
            76: [{"week_of_year": 25, "dow": 3}],
            79: [{"week_of_year": 25, "dow": 0}, {"week_of_year": 27, "dow": 0}, {"week_of_year": 34, "dow": 0}],
            80: [{"week_of_year": 23, "dow": 0}, {"week_of_year": 28, "dow": 0}],
            89: [{"week_of_year": 28, "dow": 3}, {"week_of_year": 28, "dow": 4}, {"week_of_year": 28, "dow": 5}, 
                 {"week_of_year": 28, "dow": 6}, {"week_of_year": 29, "dow": 0}],
            94: [{"week_of_year": 23, "dow": 4}, {"week_of_year": 30, "dow": 4}, {"week_of_year": 31, "dow": 0},
                 {"week_of_year": 31, "dow": 1}, {"week_of_year": 31, "dow": 2}, {"week_of_year": 31, "dow": 3},
                 {"week_of_year": 31, "dow": 4}, {"week_of_year": 33, "dow": 4}],
            95: [{"week_of_year": 28, "dow": 0}, {"week_of_year": 32, "dow": 0}],
        }
    
    for bno, rules in etc_holiday.items():
        for rule in rules:
            df.loc[
                (df['건물번호'] == bno) &
                (df['week_of_year'] == rule['week_of_year']) &
                (df['dow'] == rule['dow']),
                'is_building_holiday'
            ] = 1
    
    # 순환 인코딩
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    days_in_month = {6: 30, 7: 31, 8: 31}
    df['max_days'] = df['month'].map(days_in_month)
    
    df['day_sin'] = np.sin(2 * np.pi * ((df['month'] - 1) + (df['day'] - 1) / 31) / 12)
    df['day_cos'] = np.cos(2 * np.pi * ((df['month'] - 1) + (df['day'] - 1) / 31) / 12)
    df['month_sin'] = np.sin(2 * np.pi * (df['month'] - 1) / 12)
    df['month_cos'] = np.cos(2 * np.pi * (df['month'] - 1) / 12)
    
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)
    df['time_period'] = pd.cut(df['hour'], 
                              bins=[-1, 6, 12, 18, 24], 
                              labels=['새벽', '오전', '오후', '저녁'])
    
    # 업무시간 여부 (9-18시, 평일)
    df['business_hours'] = ((df['hour'].between(9, 18)) & 
                           (df['dow'] < 5) & 
                           (df['is_building_holiday'] == 0)).astype(int)
    df['peak_flag'] = df['hour'].between(12, 16).astype(int)
    
    return df

# 시간 피처 추가
train = add_time_features(train)
test = add_time_features(test)

In [ ]:
# 4. 결측치 처리
def safe_fillna(df):
    # 태양광/ESS 관련 결측치 처리
    df['태양광용량(kW)'] = df['태양광용량(kW)'].replace('-', 0.0).astype(float)
    df['ESS저장용량(kWh)'] = df['ESS저장용량(kWh)'].replace('-', 0.0).astype(float)
    df['PCS용량(kW)'] = df['PCS용량(kW)'].replace('-', 0.0).astype(float)
    
    # 나머지 결측치 처리
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    df[numeric_cols] = df[numeric_cols].bfill().ffill()
    df[categorical_cols] = df[categorical_cols].bfill().ffill()
    
    return df

train = safe_fillna(train)
test = safe_fillna(test)

In [ ]:
# 5. 비율·상호작용 피처
for df in (train, test):
    df['cooling_ratio'] = df['냉방면적(m2)'] / df['연면적(m2)']
    df['temp_cool_ratio'] = df['기온(°C)'] * df['cooling_ratio']

# 6. 환경 피처 추가
def add_environmental_features(df):
    """환경 기상 피처 추가"""
    T = df['기온(°C)']
    RH = df['습도(%)']
    
    # 열지수
    df['heat_index'] = (
        -8.784695
        + 1.61139411 * T
        + 2.338549 * RH
        - 0.14611605 * T * RH
        - 0.012308094 * T**2
        - 0.016424828 * RH**2
        + 0.002211732 * T**2 * RH
        + 0.00072546 * T * RH**2
        - 0.000003582 * T**2 * RH**2
    )
    
    # Cooling Degree Days
    base = 24
    df['CDD'] = (T - base).clip(lower=0)
    
    # 강수 플래그
    df['rain_flag'] = (df['강수량(mm)'] > 0).astype(int)
    
    # 불쾌지수
    df['discomfort_index'] = 9/5*T - 0.55 * (1 - RH / 100.0) * (9/5 * T - 26) + 32
    
    def discomfort_category(di):
        if di <= 68:
            return '쾌적'
        elif 68 < di <= 72:
            return '보통'
        elif 72 < di <= 77:
            return '약간 불쾌'
        elif 77 < di <= 82:
            return '불쾌'
        else:
            return '매우 불쾌'
    
    df['discomfort_category'] = df['discomfort_index'].apply(discomfort_category)
    
    # 폭염/열대야 플래그
    df['heat_wave_flag'] = (T >= 33).astype(int)
    tropical_night_condition = (((df['hour'] >= 18) | (df['hour'] <= 9)) & (T >= 25))
    df['tropical_night_flag'] = tropical_night_condition.astype(int)
    
    # 연속 고온일
    df['high_temp_flag'] = (df['기온(°C)'] >= 30).astype(int)
    
    # 바람 냉각 효과
    df['wind_chill_effect'] = df['기온(°C)'] - (df['풍속(m/s)'] * 0.5)
    
    # 기온 변화량 (전일 대비, 전시간 대비)
    df['temp_change_hour'] = df.groupby('건물번호')['기온(°C)'].diff()
    df['temp_change_day'] = df.groupby('건물번호')['기온(°C)'].diff(periods=24)
    df['temp_change_hour'] = df['temp_change_hour'].fillna(0)
    df['temp_change_day'] = df['temp_change_day'].fillna(0)
    
    # 체감온도
    def calculate_feels_like_temperature(temperature, humidity):
        """한국산업안전보건공단의 체감온도 계산"""
        Ta = temperature
        RH = humidity
        
        Tw = (Ta * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) +
              np.arctan(Ta + RH) -
              np.arctan(RH - 1.67633) +
              0.00391838 * (RH ** 1.5) * np.arctan(0.023101 * RH) - 4.686035)
        
        result = (-0.2442 + 0.55399 * Tw + 0.45535 * Ta -
                  0.0022 * (Tw ** 2) + 0.00278 * Tw * Ta + 3.0)
        
        return np.round(result, 2), np.round(Tw, 2)
    
    df['feels_like_temp'], df['wet_bulb'] = calculate_feels_like_temperature(T, RH)
    
    return df

train = add_environmental_features(train)
test = add_environmental_features(test)

In [ ]:
# 7. 시계열 피처 추가
def add_advanced_time_series_features_separate(train_df, test_df):
    """
    train과 test를 완전 분리하여 시계열 피처 생성
    각 데이터셋 내에서만 독립적으로 시계열 연산 수행 + Rolling Median 추가
    """
    
    weather_cols = ['기온(°C)', '풍속(m/s)', '습도(%)']
    col_rename_map = {
    '기온(°C)': '기온_C',
    '풍속(m/s)': '풍속_ms', 
    '습도(%)': '습도_pct'
    }
    def create_time_series_features(df, dataset_name):
        print(f"  - {dataset_name} 시계열 피처 생성...")
        df = df.sort_values(['건물번호', '일시']).copy()
        
        # 1. Differencing & Lag
        for col in weather_cols:
            col_name = col_rename_map[col]
            grouped = df.groupby('건물번호')[col]
            
            # Differencing
            df[f'{col_name}_diff1'] = grouped.diff(1)
            df[f'{col_name}_diff3'] = grouped.diff(3)
            df[f'{col_name}_diff24'] = grouped.diff(24)
            
            # Lag
            for lag in [1, 3, 24]:
                df[f'{col_name}_lag{lag}'] = grouped.shift(lag)
        
        # 2. Rolling Statistics (window=1 제외)
        for col in weather_cols:
            col_name = col_rename_map[col]
            grouped = df.groupby('건물번호')[col]
            shifted = grouped.shift(1)
            
            for w in [3, 24]:  # window=1 제거
                roll = shifted.rolling(window=w, min_periods=w)
                
                df[f'{col_name}_mean{w}'] = roll.mean()
                df[f'{col_name}_median{w}'] = roll.median()
                df[f'{col_name}_range{w}'] = roll.max() - roll.min()
                df[f'{col_name}_median_mean_diff{w}'] = roll.median() - roll.mean()
        
        return df
    
    # 각각 독립적으로 처리
    train_result = create_time_series_features(train_df.copy(), "Train")
    test_result = create_time_series_features(test_df.copy(), "Test")
    
    print(f"Train 결과: {train_result.shape}")
    print(f"Test 결과: {test_result.shape}")
    return train_result, test_result

train, test = add_advanced_time_series_features_separate(train, test)

In [ ]:
# 8. 건물별 통계
def add_building_statistical_features(train_df, test_df):  
    # Train 데이터에서만 통계 계산
    target_col = '전력소비량(kWh)'
    
    # 1. 건물별, 요일별, 시간별 전력소비량 평균
    dow_hour_mean = train.groupby(['건물번호', 'dow', 'hour'])[target_col].mean().reset_index()
    dow_hour_mean.columns = ['건물번호', 'dow', 'hour', 'dow_hour_mean']
    
    # 2. 건물별 시간별 전력소비량 평균
    hour_mean = train.groupby(['건물번호', 'hour'])[target_col].mean().reset_index()
    hour_mean.columns = ['건물번호', 'hour', 'hour_mean']
    
    # 3. 건물별 시간별 전력소비량 표준편차
    hour_std = train.groupby(['건물번호', 'hour'])[target_col].std().reset_index()
    hour_std.columns = ['건물번호', 'hour', 'hour_std']
    hour_std['hour_std'] = hour_std['hour_std'].fillna(0)  # NaN 처리
    
    # Train에 적용
    train_result = train_df.copy()
    train_result = train_result.merge(dow_hour_mean, on=['건물번호', 'dow', 'hour'], how='left')
    train_result = train_result.merge(hour_mean, on=['건물번호', 'hour'], how='left')
    train_result = train_result.merge(hour_std, on=['건물번호', 'hour'], how='left')
    
    # Test에 적용
    test_result = test_df.copy()
    test_result = test_result.merge(dow_hour_mean, on=['건물번호', 'dow', 'hour'], how='left')
    test_result = test_result.merge(hour_mean, on=['건물번호', 'hour'], how='left')
    test_result = test_result.merge(hour_std, on=['건물번호', 'hour'], how='left')
    
    # Test에서 누락된 조합에 대한 처리 (전체 평균으로 대체)
    overall_mean = train[target_col].mean()
    overall_std = train[target_col].std()
    
    test_result['dow_hour_mean'] = test_result['dow_hour_mean'].fillna(overall_mean)
    test_result['hour_mean'] = test_result['hour_mean'].fillna(overall_mean)
    test_result['hour_std'] = test_result['hour_std'].fillna(overall_std)
    
    print(f"Train 결과: {train_result.shape}")
    print(f"Test 결과: {test_result.shape}")
    
    return train_result, test_result

# 건물별 통계 피처 적용
train, test = add_building_statistical_features(train, test)

In [ ]:
# 9. 라벨 인코딩
def safe_label_encode(train_col, test_col):
    le = LabelEncoder()
    all_categories = set(train_col.astype(str)) | set(test_col.astype(str))
    le.fit(list(all_categories))
    return le.transform(train_col.astype(str)), le.transform(test_col.astype(str))

cat_cols = ['건물번호', '건물유형', 'is_holiday', 'is_building_holiday', 'is_weekday', 'time_period',
           'peak_flag', '연면적구간', '냉방면적구간', 'rain_flag', 
           'discomfort_category', 'heat_wave_flag', 'tropical_night_flag']

for col in cat_cols:
    train[col], test[col] = safe_label_encode(train[col], test[col])

In [ ]:
# 10. 건물 클러스터링 
pivot = (
    train  
    .pivot_table(index='일시', columns='건물번호', values='전력소비량(kWh)')
    .resample('D').mean()
    .T
    .bfill(axis=1).ffill(axis=1)
)

# 건물별 종합 통계
building_stats = train.groupby('건물번호').agg({
    '연면적(m2)': 'first',
    '냉방면적(m2)': 'first',
    '건물유형': 'first',
    '태양광용량(kW)': 'first',
    'ESS저장용량(kWh)': 'first',
    'PCS용량(kW)': 'first',
    '전력소비량(kWh)': 'mean',
}).fillna(0)

# 공통 건물번호만 사용
common_buildings = list(set(pivot.index) & set(building_stats.index))
pivot_filtered = pivot.loc[common_buildings]
stats_filtered = building_stats.loc[common_buildings]

# 클러스터링 실행 
ts_scaled = StandardScaler().fit_transform(pivot_filtered)
st_scaled = StandardScaler().fit_transform(stats_filtered)
X_cluster = np.hstack([ts_scaled, st_scaled])
Z = linkage(X_cluster, method='ward')
cluster_labels = fcluster(Z, t=5, criterion='maxclust')

# 클러스터 매핑
cluster_map = dict(zip(pivot_filtered.index, cluster_labels))
train['cluster_id'] = train['건물번호'].map(cluster_map)
test['cluster_id'] = test['건물번호'].map(cluster_map).fillna(0)  # 새 건물은 0으로 할당

In [ ]:
# 11. 최종 확인 및 저장
print("\n최종 결과:")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")


print(f"Features: {list(train)}")

# 결측치 최종 확인
print(f"\nTrain 결측치: {train.isnull().sum().sum()}")
print(f"Test 결측치: {test.isnull().sum().sum()}")

# 파일 저장
train.to_csv("./prep_train.csv", index=False)
test.to_csv("./prep_test.csv", index=False)
sub.to_csv('./sample_submission.csv', index=False)

print("Done.")